<a href="https://colab.research.google.com/github/stephenbeckr/randomized-algorithm-class/blob/master/Demos/demo01_exactRankR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vignette #1

APPM 5650 Randomized Algorithms, Fall 2021

Stephen Becker (original MATLAB '19, jupyter version '21) & Jake Knigge (Python '19)

**Warning**: This is not a practical algorithm, since it only works if the matrix is *exactly* low rank.

In [49]:
import numpy as np
from numpy.linalg import norm
from scipy.sparse.linalg import LinearOperator, svds

np.set_printoptions(precision = 4)      # display only four digits
rng = np.random.default_rng(12345)
n = np.int(4e3); m = n                  # dimension of problem
r = np.int(100)                         # rank of matrix

Left = rng.standard_normal( size=(m,r))
Right= rng.standard_normal( size=(r,n))
A = Left@Right
# Another case is that we *know* A has this structure, in which case we can exploit:
A_operator = LinearOperator( (m,n), matvec = lambda x : Left@(Right@x), 
                            rmatvec = lambda y : Right.T@(Left.T@y) )

def printError(U,s=None,Vh=None):
  if s is None:
    # Assume U is really the A matrix
    A_estimate = U
  else:
    S = np.reshape( s, (len(s),1) )
    A_estimate = U@(S*Vh)
  err = norm( A - A_estimate ) / norm( A )
  print(f'The error ||A-A_estimate||_F/||A||_F is {err:0.2e}')

## Find SVD of $A$ with conventional methods

Dense SVD

In [48]:
%time U, S, Vh = np.linalg.svd(A, full_matrices=False)

printError(U,S,Vh)

CPU times: user 1min 15s, sys: 2.29 s, total: 1min 17s
Wall time: 39.6 s
The error ||A-A_estimate||_F/||A||_F is 2.60e-15


Krylov subspace method (usually best for sparse matrices or some kind of structure)

In [47]:
%time U, S, Vh = scipy.sparse.linalg.svds( A, k=r)

printError(U,S,Vh)

CPU times: user 6.75 s, sys: 4.2 s, total: 11 s
Wall time: 5.61 s
The error ||A-A_estimate||_F/||A||_F is 9.24e-16


... and **if we knew the structure of $A$** :

In [46]:
%time U, S, Vh = scipy.sparse.linalg.svds( A_operator, k=r)

printError(U,S,Vh)

CPU times: user 664 ms, sys: 401 ms, total: 1.07 s
Wall time: 578 ms
The error ||A-A_estimate||_F/||A||_F is 1.31e-15


## Find SVD of $A$ with randomized method

(no knowledge of the structure of $A$ required, other than knowing a good value for $r$)

In [50]:
%%time
Omega = np.random.normal(mu, sigma, (n, r));
Y     = A@Omega       # matrix multiply
Q, R  = np.linalg.qr(Y, mode='reduced');
QtA   = Q.T@A

CPU times: user 515 ms, sys: 12.8 ms, total: 528 ms
Wall time: 279 ms


In [51]:
printError( Q@QtA )

The error ||A-A_estimate||_F/||A||_F is 4.55e-14
